In [1]:
# Imports
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import array_to_img
import zipfile

# --- GPU Setup ---
AUTO = tf.data.AUTOTUNE

# Ensure TensorFlow uses GPU (if available)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Paths for TFRecords
monet_tfrec = '/kaggle/input/gan-getting-started/monet_tfrec'
photo_tfrec = '/kaggle/input/gan-getting-started/photo_tfrec'

# Constants
IMAGE_SIZE = (256, 256)
IMAGE_SHAPE = (256, 256, 3)
BATCH_SIZE = 1
EPOCHS = 100
LAMBDA = 10

# --- Custom InstanceNorm (instead of tfa.layers.InstanceNormalization) ---
class InstanceNormalization(layers.Layer):
    def __init__(self, epsilon=1e-5, **kwargs):
        super(InstanceNormalization, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        self.gamma = self.add_weight(
            shape=(input_shape[-1],),
            initializer="ones",
            trainable=True
        )
        self.beta = self.add_weight(
            shape=(input_shape[-1],),
            initializer="zeros",
            trainable=True
        )
        super(InstanceNormalization, self).build(input_shape)

    def call(self, x):
        mean, var = tf.nn.moments(x, axes=[1, 2], keepdims=True)
        return self.gamma * (x - mean) / tf.sqrt(var + self.epsilon) + self.beta

# Function to parse TFRecords
def parse_tfrecord(example_proto):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example_proto, feature_description)
    img = tf.io.decode_jpeg(example['image'], channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = (tf.cast(img, tf.float32) / 127.5) - 1.0  # normalize [-1,1]
    return img

# Load datasets
monet_files = tf.io.gfile.glob(os.path.join(monet_tfrec, "*.tfrec"))
photo_files = tf.io.gfile.glob(os.path.join(photo_tfrec, "*.tfrec"))

monet_dataset = tf.data.TFRecordDataset(monet_files).cache()
monet_dataset = monet_dataset.map(parse_tfrecord, num_parallel_calls=AUTO)
monet_dataset = monet_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTO)

photo_dataset = tf.data.TFRecordDataset(photo_files).cache()
photo_dataset = photo_dataset.map(parse_tfrecord, num_parallel_calls=AUTO)
photo_dataset = photo_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTO)

# --- Define Models ---
def downsample(filters, size, apply_instancenorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    if apply_instancenorm:
        result.add(InstanceNormalization())
    result.add(layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same', kernel_initializer=initializer,
                                      use_bias=False))
    result.add(InstanceNormalization())
    if apply_dropout:
        result.add(layers.Dropout(0.5))
    result.add(layers.ReLU())
    return result

def Generator():
    down_stack = [
        downsample(64, 4, apply_instancenorm=False),
        downsample(128, 4),
        downsample(256, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
    ]
    up_stack = [
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]
    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(IMAGE_SHAPE[2], 4,
                                   strides=2,
                                   padding='same',
                                   kernel_initializer=initializer,
                                   activation='tanh')
    concat = layers.Concatenate()
    inputs = layers.Input(shape=IMAGE_SHAPE)
    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    skips = reversed(skips[:-1])
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = concat([x, skip])
    x = last(x)
    return Model(inputs=inputs, outputs=x, name="Generator")

def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    inputs = layers.Input(shape=IMAGE_SHAPE, name='discriminator_input')
    x = inputs
    down1 = downsample(64, 4, False)(x)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)
    zero_pad1 = layers.ZeroPadding2D()(down3)
    conv = layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer,
                         use_bias=False)(zero_pad1)
    norm1 = InstanceNormalization()(conv)
    leaky_relu = layers.LeakyReLU()(norm1)
    zero_pad2 = layers.ZeroPadding2D()(leaky_relu)
    last = layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(zero_pad2)
    return Model(inputs=inputs, outputs=last, name="Discriminator")

# --- Loss and Optimizers ---
loss_obj = keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_img, fake_img):
    real_loss = loss_obj(tf.ones_like(real_img), real_img)
    fake_loss = loss_obj(tf.zeros_like(fake_img), fake_img)
    return tf.cast(real_loss, tf.float32) + tf.cast(fake_loss, tf.float32)

def generator_loss(fake_img):
    return tf.cast(loss_obj(tf.ones_like(fake_img), fake_img), tf.float32)

def cycle_loss(real_img, cycled_img):
    return LAMBDA * tf.reduce_mean(tf.abs(tf.cast(real_img, tf.float32) - tf.cast(cycled_img, tf.float32)))

def identity_loss(real_img, same_img):
    return LAMBDA * 0.5 * tf.reduce_mean(tf.abs(tf.cast(real_img, tf.float32) - tf.cast(same_img, tf.float32)))

monet_generator_optimizer = Adam(2e-4, beta_1=0.5)
photo_generator_optimizer = Adam(2e-4, beta_1=0.5)
monet_discriminator_optimizer = Adam(2e-4, beta_1=0.5)
photo_discriminator_optimizer = Adam(2e-4, beta_1=0.5)

# Instantiate models
monet_generator = Generator()
photo_generator = Generator()
monet_discriminator = Discriminator()
photo_discriminator = Discriminator()

# --- Training Step ---
@tf.function
def train_step(real_monet, real_photo):
    real_monet = tf.cast(real_monet, tf.float32)
    real_photo = tf.cast(real_photo, tf.float32)
    
    with tf.GradientTape(persistent=True) as tape:
        fake_monet = monet_generator(real_photo, training=True)
        cycled_photo = photo_generator(fake_monet, training=True)
        fake_photo = photo_generator(real_monet, training=True)
        cycled_monet = monet_generator(fake_photo, training=True)

        same_monet = monet_generator(real_monet, training=True)
        same_photo = photo_generator(real_photo, training=True)

        disc_fake_monet = monet_discriminator(fake_monet, training=True)
        disc_real_monet = monet_discriminator(real_monet, training=True)
        disc_fake_photo = photo_discriminator(fake_photo, training=True)
        disc_real_photo = photo_discriminator(real_photo, training=True)

        monet_gen_loss = generator_loss(disc_fake_monet)
        photo_gen_loss = generator_loss(disc_fake_photo)
        total_cycle_loss = cycle_loss(real_monet, cycled_monet) + cycle_loss(real_photo, cycled_photo)
        id_loss_monet = identity_loss(real_monet, same_monet)
        id_loss_photo = identity_loss(real_photo, same_photo)

        total_monet_gen_loss = monet_gen_loss + total_cycle_loss + id_loss_monet
        total_photo_gen_loss = photo_gen_loss + total_cycle_loss + id_loss_photo

        monet_disc_loss = discriminator_loss(disc_real_monet, disc_fake_monet)
        photo_disc_loss = discriminator_loss(disc_real_photo, disc_fake_photo)

    monet_gen_grads = tape.gradient(total_monet_gen_loss, monet_generator.trainable_variables)
    photo_gen_grads = tape.gradient(total_photo_gen_loss, photo_generator.trainable_variables)
    monet_disc_grads = tape.gradient(monet_disc_loss, monet_discriminator.trainable_variables)
    photo_disc_grads = tape.gradient(photo_disc_loss, photo_discriminator.trainable_variables)

    monet_generator_optimizer.apply_gradients(zip(monet_gen_grads, monet_generator.trainable_variables))
    photo_generator_optimizer.apply_gradients(zip(photo_gen_grads, photo_generator.trainable_variables))
    monet_discriminator_optimizer.apply_gradients(zip(monet_disc_grads, monet_discriminator.trainable_variables))
    photo_discriminator_optimizer.apply_gradients(zip(photo_disc_grads, photo_discriminator.trainable_variables))

    return total_monet_gen_loss, total_photo_gen_loss, monet_disc_loss, photo_disc_loss

# --- Training loop ---
monet_iterator = iter(monet_dataset)
photo_iterator = iter(photo_dataset)

for epoch in range(EPOCHS):
    num_monet_samples = len(tf.io.gfile.glob(os.path.join(monet_tfrec, "*.tfrec")))
    num_photo_samples = len(tf.io.gfile.glob(os.path.join(photo_tfrec, "*.tfrec")))
    num_steps = min(num_monet_samples, num_photo_samples) // BATCH_SIZE
    
    for step in range(num_steps):
        try:
            monet_batch = next(monet_iterator)
            photo_batch = next(photo_iterator)
        except StopIteration:
            monet_iterator = iter(monet_dataset)
            photo_iterator = iter(photo_dataset)
            monet_batch = next(monet_iterator)
            photo_batch = next(photo_iterator)

        monet_gen_loss, photo_gen_loss, monet_disc_loss, photo_disc_loss = train_step(monet_batch, photo_batch)

    print(f"Epoch {epoch}/{EPOCHS}, Monet Gen Loss: {monet_gen_loss:.4f}, Photo Gen Loss: {photo_gen_loss:.4f}, Monet Disc Loss: {monet_disc_loss:.4f}, Photo Disc Loss: {photo_disc_loss:.4f}")

# --- Generate and save images ---
output_zip = '/kaggle/working/images.zip'
with zipfile.ZipFile(output_zip, 'w') as zipf:
    img_count = 0
    
    for batch in photo_dataset.repeat():
        if img_count >= 7000:
            break
            
        gen_imgs = monet_generator.predict(batch)
        for i in range(gen_imgs.shape[0]):
            img = (0.5 * gen_imgs[i] + 0.5) * 255.0
            img = array_to_img(img)
            img_path = f'monet_generated_{img_count}.jpg'
            img.save(img_path)
            zipf.write(img_path)
            os.remove(img_path)
            img_count += 1

print(f"Generated {img_count} images into images.zip")

2025-10-10 06:36:43.362921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760078203.637710      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760078203.714368      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available:  0


2025-10-10 06:37:02.330932: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 0/100, Monet Gen Loss: 10.7000, Photo Gen Loss: 11.9776, Monet Disc Loss: 1.7636, Photo Disc Loss: 1.8187
Epoch 1/100, Monet Gen Loss: 10.0225, Photo Gen Loss: 9.6700, Monet Disc Loss: 1.5484, Photo Disc Loss: 1.6714
Epoch 2/100, Monet Gen Loss: 7.8840, Photo Gen Loss: 7.4222, Monet Disc Loss: 1.5549, Photo Disc Loss: 1.6701
Epoch 3/100, Monet Gen Loss: 8.4237, Photo Gen Loss: 8.8908, Monet Disc Loss: 1.6029, Photo Disc Loss: 1.4329
Epoch 4/100, Monet Gen Loss: 6.6891, Photo Gen Loss: 6.5207, Monet Disc Loss: 1.5055, Photo Disc Loss: 1.4884
Epoch 5/100, Monet Gen Loss: 6.7025, Photo Gen Loss: 5.9209, Monet Disc Loss: 1.5580, Photo Disc Loss: 1.3742
Epoch 6/100, Monet Gen Loss: 10.1521, Photo Gen Loss: 11.2659, Monet Disc Loss: 1.4228, Photo Disc Loss: 1.3362
Epoch 7/100, Monet Gen Loss: 5.4972, Photo Gen Loss: 5.5039, Monet Disc Loss: 1.3774, Photo Disc Loss: 1.4491
Epoch 8/100, Monet Gen Loss: 5.1666, Photo Gen Loss: 4.8415, Monet Disc Loss: 1.3425, Photo Disc Loss: 1.3696
Epoch